In [1]:
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
BOTIFY_DATA_DIR = "C:/Users/skienbear/Desktop/Made/recsys/hw1/recsys-itmo-spring-2023/botify/data/"

# data = pd.read_json("C:/Users/skienbear/Desktop/Made/recsys/hw1/log/data.json", lines=True)[["user", "time", "track"]].copy()
data = pd.read_json("C:/Users/skienbear/Desktop/Made/recsys/hw1/log/data.json", lines=True).drop_duplicates(subset=["user", "track"])

data

,message,timestamp,user,track,time,latency,recommendation,experiments
0,next,2023-04-12 13:32:54.774,3255,2659,1.00,0.102098,1998.0,{'HW': 'C'}
1,next,2023-04-12 13:32:54.929,3255,1998,1.00,0.003442,178.0,{'HW': 'C'}
2,next,2023-04-12 13:32:54.955,3255,178,1.00,0.002904,442.0,{'HW': 'C'}
3,next,2023-04-12 13:32:54.974,3255,442,1.00,0.002299,157.0,{'HW': 'C'}
4,next,2023-04-12 13:32:54.991,3255,157,1.00,0.002177,297.0,{'HW': 'C'}
...,...,...,...,...,...,...,...,...
8779,next,2023-04-12 13:38:27.198,411,18068,0.00,0.001363,11648.0,{'HW': 'C'}
8780,next,2023-04-12 13:38:27.217,411,11648,0.03,0.001306,2816.0,{'HW': 'C'}
8781,next,2023-04-12 13:38:27.248,411,2816,1.00,0.001808,1141.0,{'HW': 'C'}
8782,next,2023-04-12 13:38:27.276,411,1141,0.80,0.002362,36438.0,{'HW': 'C'}


In [5]:
df = data[['user','track','time']]
df.columns = ['user_id', 'track_id', 'rating']

In [6]:
users = df['user_id']

In [7]:
ratings = df[['user_id', 'track_id', 'rating']]

In [8]:
tracks = pd.read_json("C:/Users/skienbear/Desktop/Made/recsys/hw1/log/tracks.json", lines=True)#.drop_duplicates(subset=["user", "track"])


In [9]:
tracks.columns = ['artist', 'track_name', 'track_id']

In [10]:
tracks = tracks[['track_id', 'artist', 'track_name']]

In [11]:
tracks

,track_id,artist,track_name
0,0,Jack Johnson,The Cove
1,1,Billy Preston,Nothing from Nothing
2,2,Paco De Lucia,Entre Dos Aguas
3,3,Josh Rouse,Under Cold Blue Stars
4,4,The Dead 60s,Riot Radio (Soundtrack Version)
...,...,...,...
49995,49995,Alain Bashung,Osez Joséphine
49996,49996,Habib Koité,Kanawa
49997,49997,De Saloon,Morder
49998,49998,She Wants Revenge,Red Flags And Long Nights


In [12]:
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

In [13]:
users_ratings = (
    ratings
    .groupby('user_id', as_index=False)
    .agg({'rating': ['count', 'mean']})
    .flatten_cols()
    .merge(users, on='user_id')
)

In [14]:
track_ratings = tracks.merge(
    ratings
    .groupby('track_id', as_index=False)
    .agg({'rating': ['count', 'mean']})
    .flatten_cols(),
    on='track_id')

In [15]:
track_ratings

,track_id,artist,track_name,rating count,rating mean
0,7,Kanye West,Stronger,2,0.500000
1,8,Jack Johnson,Constellations,4,0.700000
2,12,Lady GaGa,Paper Gangsta,6,0.456667
3,17,Harmonia,Sehr kosmisch,18,0.743333
4,29,Zero 7,Futures [Album Version],1,0.000000
...,...,...,...,...,...
4620,49966,Lou Rawls,A Natural Man,1,0.150000
4621,49974,Tiger Army,Twenty Flight Rock,1,0.570000
4622,49989,Me'Shell Ndegeocello,Outside Your Door (Album Version),1,0.350000
4623,49993,Weird Al Yankovic,Christmas At Ground Zero,2,0.720000


In [16]:
class CFModel(object):
  """Simple class that represents a collaborative filtering model"""
  def __init__(self, embedding_vars, loss, metrics=None):
    """Initializes a CFModel.
    Args:
      embedding_vars: A dictionary of tf.Variables.
      loss: A float Tensor. The loss to optimize.
      metrics: optional list of dictionaries of Tensors. The metrics in each
        dictionary will be plotted in a separate figure during training.
    """
    self._embedding_vars = embedding_vars
    self._loss = loss
    self._metrics = metrics
    self._embeddings = {k: None for k in embedding_vars}
    self._session = None

  @property
  def embeddings(self):
    """The embeddings dictionary."""
    return self._embeddings

  def train(self, num_iterations=100, learning_rate=1.0, plot_results=True,
            optimizer=tf.train.GradientDescentOptimizer):
    """Trains the model.
    Args:
      iterations: number of iterations to run.
      learning_rate: optimizer learning rate.
      plot_results: whether to plot the results at the end of training.
      optimizer: the optimizer to use. Default to GradientDescentOptimizer.
    Returns:
      The metrics dictionary evaluated at the last iteration.
    """
    
    with self._loss.graph.as_default():
      opt = optimizer(learning_rate)
      train_op = opt.minimize(self._loss)
      local_init_op = tf.group(
          tf.variables_initializer(opt.variables()),
          tf.local_variables_initializer())
      if self._session is None:
        self._session = tf.Session()
        with self._session.as_default():
          self._session.run(tf.global_variables_initializer())
          self._session.run(tf.tables_initializer())
          #tf.train.start_queue_runners()

    with self._session.as_default():
      local_init_op.run()
      iterations = []
      metrics = self._metrics or ({},)
      metrics_vals = [collections.defaultdict(list) for _ in self._metrics]

      # Train and append results.
      for i in range(num_iterations + 1):
        _, results = self._session.run((train_op, metrics))
        if (i % 10 == 0) or i == num_iterations:
          print("\r iteration %d: " % i + ", ".join(
                ["%s=%f" % (k, v) for r in results for k, v in r.items()]),
                end='')
          iterations.append(i)
          for metric_val, result in zip(metrics_vals, results):
            for k, v in result.items():
              metric_val[k].append(v)

      for k, v in self._embedding_vars.items():
        self._embeddings[k] = v.eval()

      if plot_results:
        # Plot the metrics.
        num_subplots = len(metrics)+1
        fig = plt.figure()
        fig.set_size_inches(num_subplots*10, 8)
        for i, metric_vals in enumerate(metrics_vals):
          ax = fig.add_subplot(1, num_subplots, i+1)
          for k, v in metric_vals.items():
            ax.plot(iterations, v, label=k)
          ax.set_xlim([1, num_iterations])
          ax.legend()
      return results

In [17]:
def sparse_mean_square_error(sparse_ratings, user_embeddings, movie_embeddings):
  """
  Args:
    sparse_ratings: A SparseTensor rating matrix, of dense_shape [N, M]
    user_embeddings: A dense Tensor U of shape [N, k] where k is the embedding
      dimension, such that U_i is the embedding of user i.
    movie_embeddings: A dense Tensor V of shape [M, k] where k is the embedding
      dimension, such that V_j is the embedding of movie j.
  Returns:
    A scalar Tensor representing the MSE between the true ratings and the
      model's predictions.
  """
  predictions = tf.gather_nd(
      tf.matmul(user_embeddings, movie_embeddings, transpose_b=True),
      sparse_ratings.indices)
  loss = tf.losses.mean_squared_error(sparse_ratings.values, predictions)
  return loss

In [18]:
def build_rating_sparse_tensor(ratings_df):
  """
  Args:
    ratings_df: a pd.DataFrame with `user_id`, `movie_id` and `rating` columns.
  Returns:
    a tf.SparseTensor representing the ratings matrix.
  """
  indices = ratings_df[['user_id', 'track_id']].values
  values = ratings_df['rating'].values
  return tf.SparseTensor(
      indices=indices,
      values=values,
      dense_shape=[users.shape[0], tracks.shape[0]])

In [19]:
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

In [20]:
def build_model(ratings, embedding_dim=3, init_stddev=1.):
  """
  Args:
    ratings: a DataFrame of the ratings
    embedding_dim: the dimension of the embedding vectors.
    init_stddev: float, the standard deviation of the random initial embeddings.
  Returns:
    model: a CFModel.
  """
  # Split the ratings DataFrame into train and test.
  train_ratings, test_ratings = split_dataframe(ratings)
  # SparseTensor representation of the train and test datasets.
  A_train = build_rating_sparse_tensor(train_ratings)
  A_test = build_rating_sparse_tensor(test_ratings)
  # Initialize the embeddings using a normal distribution.
  U = tf.Variable(tf.random.normal(
      [A_train.dense_shape[0], embedding_dim], stddev=init_stddev))
  V = tf.Variable(tf.random.normal(
      [A_train.dense_shape[1], embedding_dim], stddev=init_stddev))
  train_loss = sparse_mean_square_error(A_train, U, V)
  test_loss = sparse_mean_square_error(A_test, U, V)
  metrics = {
      'train_error': train_loss,
      'test_error': test_loss
  }
  embeddings = {
      "user_id": U,
      "track_id": V
  }
  return CFModel(embeddings, train_loss, [metrics])

In [21]:
model = build_model(ratings, embedding_dim=20, init_stddev=0.5)

In [22]:
model.train(num_iterations=1000, learning_rate=5.)

InvalidArgumentError: Graph execution error:

Detected at node 'GatherNd' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda3\lib\asyncio\base_events.py", line 596, in run_forever
      self._run_once()
    File "C:\Pro